In [74]:
from sklearn.model_selection import train_test_split
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import matthews_corrcoef, accuracy_score




In [11]:
df = pd.read_csv('./final_data2.csv')

In [63]:
np.random.seed(42)
val_plays = np.random.choice(df['game_play'].unique(), size=2)
val_set = df.apply(lambda row: row[df['game_play'].isin(val_plays)])
train_set = df.apply(lambda row: row[~df['game_play'].isin(val_plays)])

In [ ]:
categorical_data_for_fitting = [
    ['home', 'CB', 'home', 'CB', ],
    ['home', 'DE', 'home', 'DE', ],
    ['home', 'FS', 'home', 'FS', ],
    ['home', 'TE', 'home', 'TE', ],
    ['home', 'ILB', 'home', 'ILB', ],
    ['home', 'OLB', 'home', 'OLB', ],
    ['home', 'T', 'home', 'T', ],
    ['home', 'G', 'home', 'G', ],
    ['home', 'C', 'home', 'C', ],
    ['home', 'QB', 'home', 'QB', ],
    ['home', 'WR', 'home', 'WR', ],
    ['home', 'RB', 'home', 'RB', ],
    ['home', 'NT', 'home', 'NT', ],
    ['home', 'DT', 'home', 'DT', ],
    ['home', 'MLB', 'home', 'MLB', ],
    ['home', 'SS', 'home', 'SS', ],
    ['home', 'OT', 'home', 'OT', ],
    ['home', 'LB', 'home', 'LB', ],
    ['home', 'OG', 'home', 'OG', ],
    ['home', 'SAF', 'home', 'SAF', ],
    ['home', 'DB', 'home', 'DB', ],
    ['home', 'LS', 'home', 'LS', ],
    ['home', 'K', 'home', 'K', ],
    ['home', 'P', 'home', 'P', ],
    ['home', 'FB', 'home', 'FB', ],
    ['home', 'S', 'home', 'S', ],
    ['home', 'DL', 'Ground', 'DL', ],
    ['away', 'HB', 'away', 'HB', ],
    ['away', 'HB', 'away', 'Ground', ],
]
one_hot = OneHotEncoder()
one_hot.fit(categorical_data_for_fitting)

OneHotEncoder()

In [64]:
def get_features(df):
       features_motion = df[['rel_pos_x',
              'rel_pos_y', 'rel_pos_mag', 'rel_pos_ori', 'rel_speed_x', 'rel_speed_y',
              'rel_speed_mag', 'rel_speed_ori', 'rel_acceleration_x',
              'rel_acceleration_y', 'rel_acceleration_mag', 'rel_acceleration_ori', 'orientation_1', 'orientation_2', 'G_flug']].fillna(0)

       features_team = df[['team_1', 'position_1','team_2', 'position_2']].fillna('Ground').to_numpy()
       features_team = one_hot.transform(features_team).toarray()


       features = np.hstack((features_motion, features_team))*1.0
       labels = df['contact'].to_numpy()
       return features, labels

In [65]:
trainX, trainY = get_features(train_set)
valX, valY = get_features(val_set)

In [76]:
xgb_cl = xgb.XGBClassifier(tree_method='gpu_hist', n_jobs=4)

# Fit
xgb_cl.fit(trainX, trainY)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=0,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=4, num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [77]:
val_preds = xgb_cl.predict(valX)

In [78]:
matthews_corrcoef(valY, val_preds)

0.5620691324557482